In [1]:
from datascience import *
import numpy as np
from math import *
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

## Lesson 24: Hypothesis Testing Errors & Power

Throughout this block, we have been studying hypothesis tests. We have covered the four basic steps of any hypothesis test, and we have practiced various methods for obtaining the distribution of our test statistic under the null hypothesis. 

After we have reached a conclusion (reject or fail to reject), we must consider possible errors. 

### Type I error 

Type I error is the event that we rejected the null hypothesis when the null hypothesis was actually true. Type I error is also known as a false positive. The probability of a Type I error is usually defined by the threshold used for rejection. A common threshold is 0.05. Those of you who have taken statistics before may recognize this value as $\alpha$. 

### Type II error

Type II error is the event that we failed to reject the null hypothesis when the null hypothesis was actually false. This is otherwise known as a false negative. The probability of a Type II error is harder to find and requires a more in-depth analysis of a hypothesis test. The probability of a Type II error is often given as $\beta$, and $1-\beta$ is referred to as **Power**. The power of a test is probability that we will reject the null hypothesis when we are supposed to. 

Which one of these errors is more serious? It depends on the context of the problem. 

### Example: Golf Balls

Joe has a summer job at a golf course and one of his jobs is to fish out golf balls from the water traps. He has a theory that certain types of golf ball are more likely to end up in the water than others. Let's assume there are four brands of golf ball, let's and assume that all four are used equally at this golf course. He fishes out 100 golf balls and counts each brand. He finds 30 of brand A, 30 of brand B, 20 of brand C and 20 of brand D. Conduct a hypothesis test to determine whether certain types of golf ball are more likely than others to end up in the water.

Step 1: Hypotheses

Each type of gold ball is equally likely to end up in the water

Step 2: Test statistic

There are many correct answers, but let's go with sum of absolute difference between observed and expected counts under $H_0$. To do this, we need to find the expected counts. If each ball was equally likely, how many should we expected to find of each if we select 100 golf balls? 

In [2]:
expected = make_array(25, 25, 25, 25)
expected

array([25, 25, 25, 25])

Step 3: $p$-value

We need the distribution of the test statistic under $H_0$. 

In [3]:
test = make_array(30, 30, 20, 20)
test = np.sum(np.abs(test - expected))
test

20

In [4]:
null_table = Table().with_column('Type', make_array('A', 'B', 'C', 'D')).with_column('p', make_array(.25, .25, .25, .25))
types_only = null_table.drop('p')
avg = make_array()
rep = 10000
i = 0
while(i < rep):
    sample = np.sum(np.abs(types_only.sample(100, with_replacement=True).group('Type')['count'] - expected))
    avg = np.append(avg, sample)
    i += 1
p_of_test = np.count_nonzero(avg >= test)/rep
p_of_test

0.178

Step 4: Conclude

Since there is about an 18.7% chance based on our random sampling that the results obtained are in accordance with the null hypothesis, we fail to reject the null hypothesis.

What kind of error could we have made in this case? 

We could have made a type II error by failing to reject.

#### Power 
Suppose that, in truth, 30% of the balls found in the water were brand A, 30% were brand B, 20% were brand C and 20% were brand D. In this case, our collected sample reflected this truth perfectly. However, our hypothesis test failed to recognize this deviation from equal proportions. We made a type II error. This is because this test has fairly low power. Use simulation to determine the power of this test. 

I am looking for the probability that I reject the null hypothesis given the true proportions laid out above. Well, first I need to figure out for what values of my test statistic I would reject $H_0$. 

In [10]:
print(percentile(95, avg)) # find our bound
print(np.count_nonzero(avg >= 25)/rep)

24.0
0.0435


Next, I need to simulate from the true population and determine how often my test statistic would have met this threshold. 

In [12]:
power_sample = make_array()
for i in range(rep):
    trial = stats.multinomial.rvs(100, [.3, .3, .2, .2], size=1)
    power_sample = np.append(power_sample, np.sum(abs(trial - 25)))
power_sample

array([34., 22., 28., ..., 12., 38., 22.])

In [13]:
np.count_nonzero(power_sample >= 25)/rep

0.3504

What do you think about this power? 

Seems fairly low.  We ought to simulate more

Repeat this power calculation, but assume Joe collects 500 balls instead of 100. Note that you will have to obtain a new critical value. What does this tell you about power and sample size?

In [21]:
# repeat the experiment with 500 instead of 100 - but use scipy this time because its faster
null_table = Table().with_column('Type', make_array('A', 'B', 'C', 'D')).with_column('p', make_array(.25, .25, .25, .25))
types_only = null_table.drop('p')
avg = make_array()
rep = 10000
i = 0
while(i < rep):
    sample = stats.multinomial.rvs(500, [.25, .25, .25, .25], size=1)
    avg = np.append(avg, np.sum(abs(sample-125))) #125 is expected val
    i += 1
sample

array([[117, 122, 125, 136]])

In [24]:
print(percentile(95, avg)) # find our bound
print(np.count_nonzero(avg >= 55)/rep)

power_sample = make_array()
for i in range(rep):
    trial = stats.multinomial.rvs(500, [.3, .3, .2, .2], size=1)
    power_sample = np.append(power_sample, np.sum(abs(trial - 125)))
print(power_sample)
print(np.count_nonzero(power_sample >= 55)/rep)

54.0
0.0472
[108.  74.  84. ... 108. 120. 108.]
0.9814


The power is now 98% which is much stronger.  

...